In [ ]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import os
import pandas as pd
import math

In [ ]:
stopword = set(stopwords.words('english'))
# stopword

In [ ]:
data = pd.read_csv("dataset.csv", delimiter=",")

In [7]:
X = list(data["news"])
y = list(data['type'])

In [8]:
import string
def clear_punctuation(s):
    clear_string = ""
    for symbol in s:
        if symbol not in string.punctuation:
            clear_string += symbol
    return clear_string

In [9]:
index = 0
clean = []
for line in X:
    line = clear_punctuation(line)
    line = line.strip("/r/n")
    array = line.split(' ')
    replaced = ''
    for word in array:
        if word.lower().strip() not in stopword:
            replaced += word+' '
    if replaced!='':
        replaced=replaced[0:len(replaced)-1]
        clean.append(index)
    X[index] = replaced
    index+=1
X_new = []
y_new = []
for i in clean:
    X_new.append(X[i])
    y_new.append(y[i])

In [10]:
stemmer = SnowballStemmer('english')

In [11]:
index = 0
for line in X_new:
    array = line.split(' ')
    replaced = ''
    for word in array:
        replaced += stemmer.stem(word)+' '
    if replaced!='':
        replaced=replaced[0:len(replaced)-1]
    X_new[index] = replaced
    index+=1

In [12]:
test = X_new[0].split(" ")
label = y_new[0]
X_new.pop(0)
y_new.pop(0)
# test,label

'business'

In [13]:
vocabulary = set()
for line in X_new:
    for word in line.split(' '):
        vocabulary.add(word)

In [14]:
V = len(vocabulary)
V

28573

In [16]:
business, entertainment, politics, sport, tech = 0,0,0,0,0
for i in range(len(X_new)):
    if y_new[i] == 'business':
        business+=len(X_new[i].split(" "))
    elif y_new[i] == 'entertainment':
        entertainment+=len(X_new[i].split(" "))
    elif y_new[i] == 'politics':
        politics+=len(X_new[i].split(" "))
    elif y_new[i] == 'sport':
        sport+=len(X_new[i].split(" "))
    elif y_new[i] == 'tech':
        tech+=len(X_new[i].split(" "))
business, entertainment, politics, sport, tech

(104704, 78313, 110225, 98152, 118452)

In [17]:
b, e, p, s, t = 0,0,0,0,0
for label in y_new:
    if label.strip() == 'business':
        b+=1
    elif label.strip() == 'entertainment':
        e+=1
    elif label.strip() == 'politics':
        p+=1
    elif label.strip() == 'sport':
        s+=1
    elif label.strip() == 'tech':
        t+=1
prior_business = b/(b+e+p+s+t)
prior_entertainment = e/(b+e+p+s+t)
prior_politics = p/(b+e+p+s+t)
prior_sport = s/(b+e+p+s+t)
prior_tech = t/(b+e+p+s+t)
prior_business,prior_entertainment,prior_politics,prior_sport,prior_tech

(0.2289698605488079,
 0.17363922627080522,
 0.18758434547908232,
 0.22941970310391363,
 0.18038686459739092)

In [18]:
business_probs = []
entertainment_probs = []
politics_probs = []
sport_probs = []
tech_probs = []
for inp_word in test:
    count_business_word = 0
    count_entertainment_word = 0
    count_politics_word = 0
    count_sport_word = 0
    count_tech_word = 0
    for i in range(len(X_new)):
        if y_new[i] == 'business':
            count_business_word += X_new[i].split(" ").count(inp_word)
        elif y_new[i] == 'entertainment':
            count_entertainment_word += X_new[i].split(" ").count(inp_word)
        elif y_new[i] == 'politics':
            count_politics_word += X_new[i].split(" ").count(inp_word)
        elif y_new[i] == 'sport':
            count_sport_word += X_new[i].split(" ").count(inp_word)
        elif y_new[i] == 'tech':
            count_tech_word += X_new[i].split(" ").count(inp_word)
    business_probs.append((count_business_word+1)/(business+V))
    entertainment_probs.append((count_entertainment_word+1)/(entertainment+V))
    politics_probs.append((count_politics_word+1)/(politics+V))
    sport_probs.append((count_sport_word+1)/(sport+V))
    tech_probs.append((count_tech_word+1)/(tech+V))

In [19]:
prob_is_business = math.log(prior_business) 
prob_is_entertainment = prior_entertainment
prob_is_politics = prior_politics
prob_is_sport = prior_sport
prob_is_tech = prior_tech
for i in range(len(business_probs)):
    prob_is_business *= math.log(business_probs[i])
    prob_is_entertainment *= math.log(entertainment_probs[i])
    prob_is_politics *= math.log(politics_probs[i])
    prob_is_sport *= math.log(sport_probs[i])
    prob_is_tech *= math.log(tech_probs[i])

In [20]:
prob_is_business,prob_is_entertainment,prob_is_politics,prob_is_sport,prob_is_tech

(2.6784818190655337e+176,
 -2.43664494206232e+189,
 -7.88486429392349e+184,
 -4.717241220662203e+190,
 -1.3320298894620442e+185)